# Lecture 3: Geometry of Linear Programming and Simplex Method
## Contents
- [Wyndor's Production Problem](#section1)
    - [Naive implementation](#subsection1.1)
    - [Separating model and data](#subsection1.2)
    - [Loading data from .csv file](#subsection1.3)

## Wyndor's Production Problem <a id="section1"></a>
Recall the linear programming formulation:
\begin{equation}
\begin{split}
\max~ & 3x_1 + 5x_2 \\
s.t. ~& x_1 \leq 4\\
& 2x_2\leq 12 \\
& 3x_1 + 2x_2 \leq 18\\
& x_1,x_2\geq 0.
\end{split}
\end{equation}
### Naive implementation <a id="subsection1.1"></a>
One first define the "model" object in Gurobi. Then all the functions inside the Gurobi model object can be used to build up the mathematical model. Please go to https://www.gurobi.com/documentation/9.0/refman/py_model.html to check the list of functions and their usage. 

In [1]:
from gurobipy import *

m = Model("Wyndor")

# Creat variables
# addVar(lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="", column=None)
# lb: lower bound, ub: upper bound
# vtype: continuous, binary or integer
# name: name for the variable
x1 = m.addVar(name = "Doors")
x2 = m.addVar(name = "Windows")

# Set objective
# setObjective ( expr, sense=None )
# expr: linear or quadratic expression
# sense: GRB.MINIMIZE or GRB.MAXIMIZE
m.setObjective(3*x1 + 5*x2, GRB.MAXIMIZE)

# Add constraint: 
m.addConstr(x1 <= 4, "Plant1")

m.addConstr(2*x2 <= 12, "Plant2")

m.addConstr(3*x1 + 2*x2 <= 18, "Plant3")

# Solving the model
m.optimize()

# Print optimal solutions and optimal value
print('-------------------')
print(x1, x2)
print(x1.VarName, x1.x)
print(x2.VarName, x2.x)
    
print('Obj:', m.objVal)


Academic license - for non-commercial use only
Optimize a model with 3 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  3.600000000e+01
-------------------
<gurobi.Var Doors (value 2.0)> <gurobi.Var Windows (value 6.0)>
Doors 2.0
Windows 6.0
Obj: 36.0


In [2]:
type(x1), type(x1.VarName), type(x1.x)

(gurobipy.Var, str, float)

### Separating model and data <a id="subsection1.2"></a>

In [3]:
from gurobipy import *
import numpy as np

#########Parameters Set-up############

# Objective coefficient: profit for each product
profit = np.array([3, 5])
print(profit)
print(type(profit))
print(profit.shape)

[3 5]
<class 'numpy.ndarray'>
(2,)


In [4]:
# Constraint right-hand-side: capacity for each plant
capacity = np.array([4, 12, 18])

# A matrix: the consumption of capacity at plant i of product j
rate = np.array([[1, 0],
                 [0, 2], 
                 [3, 2]])

print(rate)
print(rate.shape)

[[1 0]
 [0 2]
 [3 2]]
(3, 2)


In [5]:
# From A matrix, extract the number of products: N and the number of plants: M
M, N = rate.shape

print("M = %g,  N = %g" % (M, N) )

M = 3,  N = 2


In [11]:
#########Model Set-up###############

m = Model("Wyndor")

# Creat variables
# addVars ( *indices, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="" )
# indices: can be one or more integer values, e.g.,  x = model.addVars(2), which creates x[0], x[1]; 
#          e.g., x = model.addVars(2, 3), which creates x[0,0],x[0,1],x[0,2],x[1,0],x[1,1],x[1,2];
#          can be a list of tuples, e.g., x = model.addVars([(0,0), (1,1), (2,2)]), which creates x[0,0],x[1,1],x[2,2].

x = m.addVars(N, name = "x")

# Set objective
m.setObjective( quicksum(profit[i]*x[i] for i in range(N)), GRB.MAXIMIZE)

# Add constraints: 
m.addConstrs(( quicksum(rate[i,j]*x[j] for j in range(N)) <= capacity[i] for i in range(M) ), "Plant")
#Alternatively, one can also add constraints one by one explicitly in a for-loop:
# for i in range(M):
#     m.addConstr( quicksum(rate[i,j]*x[j] for j in range(N)) <= capacity[i] )

# Solving the model
m.optimize()

# Print optimal solutions and optimal value
print('-------------------------------------')

for v in m.getVars():
    print(v.VarName, v.x)
    
print('Obj:', m.objVal)

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Found heuristic solution: objective 27.0000000
Presolve removed 3 rows and 2 columns
Presolve time: 0.05s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.12 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 36 27 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.600000000000e+01, best bound 3.600000000000e+01, gap 0.0000%
-------------------------------------
x[0] 2.0
x[1] 6.0
Obj: 36.0


### Loading data from .csv file <a id="subsection1.3"></a>

In [7]:
from gurobipy import *
import numpy as np
import pandas as pd


#########Parameters Set-up############
# Read the data from the csv file and use the first column as the index of rows
Wyndor_data = pd.read_csv('Wyndor.csv', index_col = 0)
print(Wyndor_data)
print(Wyndor_data.shape)

# Record the number of rows and columns in the data
temp_M, temp_N = Wyndor_data.shape


                  Doors  Windows  Capacity
Profit per batch      3        5       NaN
Plant 1               1        0       4.0
Plant 2               0        2      12.0
Plant 3               3        2      18.0
(4, 3)


In [8]:
#One is referred to http://pandas.pydata.org/pandas-docs/stable/indexing.html 
#for indexing and selecting data for the dataframe object in pandas.

# Extracting objective coefficient via selection by position: .iloc
# When slicing the data, the start bound is included, while the end bound is excluded
profit = Wyndor_data.iloc[0, 0:temp_N-1]
print(profit) 
print('Index of the data:')
print(profit.index) 
print('Values of the data:')
print(profit.values)

#Extracting the values by ignoring the index and header of the dataframe
profit = profit.values

Doors      3.0
Windows    5.0
Name: Profit per batch, dtype: float64
Index of the data:
Index(['Doors', 'Windows'], dtype='object')
Values of the data:
[3. 5.]


In [9]:
#Constraint right-hand-side: capacity for each plant
capacity = Wyndor_data.iloc[1:temp_M, temp_N-1]
capacity = capacity.values

#A matrix: the consumption of capacity at plant i of product j
rate = Wyndor_data.iloc[1:temp_M, 0:temp_N-1]
print(rate)
rate = rate.values
print(rate)

#From A matrix, extract the number of products: N and the number of plants: M
M, N = rate.shape


         Doors  Windows
Plant 1      1        0
Plant 2      0        2
Plant 3      3        2
[[1 0]
 [0 2]
 [3 2]]


In [10]:
#########Model Set-up###############

m = Model("Wyndor")

# Creat variables
# addVars ( *indices, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="" )
x = m.addVars(N, name = 'Product')

# Set objective
m.setObjective( quicksum(profit[i]*x[i] for i in range(N)), GRB.MAXIMIZE)

# Add constraints: 
m.addConstrs(( quicksum(rate[i,j]*x[j] for j in range(N)) <= capacity[i] for i in range(M) ), name = "Plant")

# Solving the model
m.optimize()

#  Print optimal solutions and optimal value
for v in m.getVars():
    print(v.VarName, v.x)
    
print('Obj:', m.objVal)


Optimize a model with 3 rows, 2 columns and 4 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.5000000e+01   1.500000e+00   0.000000e+00      0s
       1    3.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  3.600000000e+01
Product[0] 2.0
Product[1] 6.0
Obj: 36.0
